In [1]:
import cv2
import json
import time
import threading
import queue
import sys
import os
import pathlib

import numpy as np
import matplotlib as plt

from PIL import Image
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from os import listdir
from os.path import isfile, join

In [2]:
### REQUIRED ###

# PROJECT NAMES SHOULD NOT USE UNDERSCORES AS THEY ARE USED TO SEPARATE SAMPLES INTO THEIR CATEGORIES HERE

# define location of dataset and return all files
dataset_location = "I:/FARTS/LATEST/Leptoglossus-combined"
target_dir = "I:/mmsegmentation/data/Leptoglossus-combined/"

### OPTIONAL ###

# set True to show processing results for each image (disables parallel processing)
DEBUG = False

# number of pixels added to pad masks to avoid cutting off contours
mask_padding = 0

enforce_single_class = False # overwrites multiple classes and groups all instances as one

# determine the proportion of a bounding box that needs to be filled before considering the visibility as too low
visibility_threshold = 0.1

# test split (what split is witheld)
amount_test = 0.01

In [3]:
all_files = [f for f in listdir(dataset_location) if isfile(join(dataset_location, f))]
all_files.sort()

# next, sort files into images, depth maps, segmentation maps, data, and colony info
# we only need the location and name of the data files, as all passes follow the same naming convention
dataset_data = []
dataset_img = []
dataset_ID = []
dataset_depth = []
dataset_norm = []
dataset_colony = None

for file in all_files:
    loc = dataset_location + "/" + file
    file_info = file.split("_")
    
    if file_info[1] == "BatchData":
        dataset_colony = loc
        
    elif len(file_info) == 2:
        # images are available in various formats, but annotation data is always written as json files
        if file_info[-1].split(".")[-1] == "json":
            dataset_data.append(loc)
        else:
            dataset_img.append(loc)
            
    elif file_info[-1].split(".")[0] == "ID":
        dataset_ID.append(loc)
    elif file_info[-1].split(".")[0]  == "depth":
        dataset_depth.append(loc)
    elif file_info[-1].split(".")[0]  == "norm":
        dataset_norm.append(loc)
        
print("Found",len(dataset_data),"samples...")

# next sort the colony info into its IDs to determine the colony size and individual scales
# Opening colony (BatchData) JSON file
colony_file = open(dataset_colony)
 
# returns JSON object as a dictionary
colony = json.load(colony_file)
colony_file.close()


""" !!! requires IDs, model names, scales !!! """

if not enforce_single_class:
    # get provided classes to create a dictionary of class IDs and class names
    all_classes = []
    for subject in colony["Subject Variations"]:
        all_classes.append(colony["Subject Variations"][subject]["Class"])
        
    subject_class_names = []
    for class_name in all_classes:
        # check if exists in unique_list or not and replace any spaces with underscores
        class_name = class_name.replace(" ", "_")
        if class_name not in subject_class_names:
            # append unique classes 
            subject_class_names.append(class_name)
        
    subject_classes = {}
    for id,sbj in enumerate(subject_class_names):
        subject_classes[str(sbj)] = id
else:
    subject_class_names = ["insect_0"] #np.array([int(0)], dtype=int)
    subject_classes = {"insect" : 0}

print("\nA total of",len(subject_class_names),"unique classes have been found.")
print("The classes and respective class IDs are:\n",subject_classes,"\n")


print("Loaded colony file with seed", colony['Seed']) #,"and",len(colony['ID']),"individuals.")
    
if len(colony['Subject Variations']) > 1:
    multi_animal = True
    print("Generating MULTI-animal dataset! Containing",len(colony['Subject Variations']),"individuals")
else:
    multi_animal = False
    print("Generating SINGLE-animal dataset!")

Found 20000 samples...

A total of 1 unique classes have been found.
The classes and respective class IDs are:
 {'Leptoglossus_zonatus': 0} 

Loaded colony file with seed 987654321
Generating MULTI-animal dataset! Containing 15 individuals


As there may be animals for which we don't use all bones we can return a list of all labels and exclude the respective locations from the pose data. As all animals use the same convention, we can simply read in one example and remove the corresponding indices from all animals.

In [4]:
### REQUIRED ###
# specify which labels to ignore. By default, all keypoints are written into the dataset
# in this example we omit all keypoints relating to wings. Refer to the base_rig documentation for naming conventions
omit_labels = ['w_1_l', 'w_1_l_end', 'w_2_l', 'w_2_l_end', 'w_1_r', 'w_1_r_end', 'w_2_r', 'w_2_r_end', 'root']

# for simplicity we'll assume that at this stage all subjects use the same armature and therefore report the same keypoints
# we therefore load the first sample from the list and find the subjects keypoint hierarchy
sample_file = open(dataset_data[0])

# returns JSON object as a dictionary
sample = json.load(sample_file)
sample_file.close()

first_entry_key = list(sample["iterationData"]["subject Data"][0].keys())[0]
labels = list(sample["iterationData"]["subject Data"][0][first_entry_key]["keypoints"].keys())

# show all used labels:
print("\nAll labels:  ",labels)

print("\nOmitting labels:  ", omit_labels)

# removing all occurences of omitted labels from the labels list to be used as keys below
labels = [x for x in labels if x not in omit_labels]

print("\nUsing labels:  ", labels)


All labels:   ['b_t', 'b_a_1', 'b_a_2', 'b_a_3', 'b_a_4', 'b_a_5', 'l_1_co_r', 'l_1_tr_r', 'l_1_fe_r', 'l_1_ti_r', 'l_1_ta_r', 'l_1_pt_r', 'l_2_co_r', 'l_2_tr_r', 'l_2_fe_r', 'l_2_ti_r', 'l_2_ta_r', 'l_2_pt_r', 'l_3_co_r', 'l_3_tr_r', 'l_3_fe_r', 'l_3_ti_r', 'l_3_ta_r', 'l_3_pt_r', 'w_1_r', 'w_2_r', 'l_1_co_l', 'l_1_tr_l', 'l_1_fe_l', 'l_1_ti_l', 'l_1_ta_l', 'l_1_pt_l', 'l_2_co_l', 'l_2_tr_l', 'l_2_fe_l', 'l_2_ti_l', 'l_2_ta_l', 'l_2_pt_l', 'l_3_co_l', 'l_3_tr_l', 'l_3_fe_l', 'l_3_ti_l', 'l_3_ta_l', 'l_3_pt_l', 'w_1_l', 'w_2_l', 'b_h', 'ma_r', 'an_1_r', 'an_2_r', 'an_3_r', 'ma_l', 'an_1_l', 'an_2_l', 'an_3_l']

Omitting labels:   ['w_1_l', 'w_1_l_end', 'w_2_l', 'w_2_l_end', 'w_1_r', 'w_1_r_end', 'w_2_r', 'w_2_r_end', 'root']

Using labels:   ['b_t', 'b_a_1', 'b_a_2', 'b_a_3', 'b_a_4', 'b_a_5', 'l_1_co_r', 'l_1_tr_r', 'l_1_fe_r', 'l_1_ti_r', 'l_1_ta_r', 'l_1_pt_r', 'l_2_co_r', 'l_2_tr_r', 'l_2_fe_r', 'l_2_ti_r', 'l_2_ta_r', 'l_2_pt_r', 'l_3_co_r', 'l_3_tr_r', 'l_3_fe_r', 'l_3_ti_r', 'l

Create required output folders used in the mmlab segmentation conventions
- **/images/** (containing a copy of the original input images)
- **/annotations/** (containing the segmentation maps)
- **/imageLists/**(containing three txt files which contain the names of the images separated into all.txt / test.txt / train.txt

In [5]:
# create output folders
output_folders = ["images", "annotations", "imageLists"]
for i, f in enumerate(output_folders):
    if not os.path.exists(target_dir + "/" + f):
        os.mkdir(target_dir + "/" + f)
        
"""
    if i < 2:
        if not os.path.exists(target_dir + "/" + f + "/train"):
            os.mkdir(target_dir + "/" + f + "/train")
        if not os.path.exists(target_dir + "/" + f + "/test"):
            os.mkdir(target_dir + "/" + f + "/test")
"""

'\n    if i < 2:\n        if not os.path.exists(target_dir + "/" + f + "/train"):\n            os.mkdir(target_dir + "/" + f + "/train")\n        if not os.path.exists(target_dir + "/" + f + "/test"):\n            os.mkdir(target_dir + "/" + f + "/test")\n'

Create a test and a train image list, according to the specified split.
When writing out the images, automatically plcae them into their respective folders.

In [6]:
imageLists_all = target_dir + "/imageLists/all.txt"
imageLists_train = target_dir + "/imageLists/train.txt"
imageLists_test = target_dir + "/imageLists/test.txt"

imageLists_all_orig = [img.split('/')[-1][:-4] + "_synth" for img in dataset_img]

imageLists_all_orig_shuffle = shuffle(imageLists_all_orig, random_state=0)
num_train_examples = int(np.floor(len(imageLists_all_orig_shuffle) * (1 - amount_test)))

print("Using", num_train_examples, "training images and",
      int(np.floor(len(imageLists_all_orig_shuffle) - (len(imageLists_all_orig_shuffle) * (1 - amount_test)))), "test images. (" + str(amount_test * 100),
      "%)")

files_train = imageLists_all_orig_shuffle[0:num_train_examples]
files_test = imageLists_all_orig_shuffle[num_train_examples:]

with open(imageLists_all, 'w') as f:
    for line in imageLists_all_orig_shuffle:
        f.write(line)
        f.write('\n')
        
with open(imageLists_train, 'w') as f:
    for line in files_train:
        f.write(line)
        f.write('\n')
        
with open(imageLists_test, 'w') as f:
    for line in files_test:
        f.write(line)
        f.write('\n')

Using 19800 training images and 200 test images. (1.0 %)


In [7]:
# transform between sRGB and linear colour space (optional)

def to_linear(srgb):
    linear = np.float32(srgb) / 255.0
    less = linear <= 0.04045
    linear[less] = linear[less] / 12.92
    linear[~less] = np.power((linear[~less] + 0.055) / 1.055, 2.4)
    return linear * 255.0

    
def from_linear(linear):
    srgb = linear.copy()
    less = linear <= 0.0031308
    srgb[less] = linear[less] * 12.92
    srgb[~less] = 1.055 * np.power(linear[~less], 1.0 / 2.4) - 0.055
    return srgb * 255.0

In [8]:
def getThreads():
    """ Returns the number of available threads on a posix/win based system """
    if sys.platform == 'win32':
        return int(os.environ['NUMBER_OF_PROCESSORS'])
    else:
        return int(os.popen('grep -c cores /proc/cpuinfo').read())

class exportThread(threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q

    def run(self):
        print("Starting " + self.name)
        process_detections(self.name, self.q)
        print("Exiting " + self.name)
        
def createThreadList(num_threads):
    threadNames = []
    for t in range(num_threads):
        threadNames.append("Thread_" + str(t))

    return threadNames

def process_detections(threadName, q):
    while not exitFlag_export:
        queueLock.acquire()
        if not workQueue_export.empty():
            
            data_input = q.get()
            i, data_loc, img, ID = data_input
            queueLock.release()
            
            display_img = cv2.imread(img)
            display_img_orig = display_img.copy()
            
            # compute visibility for each individual
            seg_img = cv2.imread(ID)
            seg_img_display = seg_img.copy()
            
            data_file = open(data_loc)
            # returns JSON object as a dictionary
            data = json.load(data_file)
            data_file.close()
            
            img_shape = display_img.shape
            
            output_mask = np.zeros(img_shape[:2], dtype=np.uint8)
            
            # only add images that contain visibile individuals
            is_empty = True
            img_info = []
            
            # check if the size of the image and segmentation pass match
            if display_img.shape != seg_img.shape:
                print("Size mismatch of image and segmentation pass for sample",data_input[1].split("/")[-1],"!")
            else:
                for im, individual in enumerate(data["iterationData"]["subject Data"]):
                    ind_key = list(individual.keys())[0]
                    ind_ID = int(ind_key)
                    # WARNING ID numbering begins at 1

                    fontColor = (int(ID_colours[ind_ID,0]),
                                 int(ID_colours[ind_ID,1]),
                                 int(ID_colours[ind_ID,2]))
                    
                    bbox_orig = [individual[ind_key]["2DBounds"]["xmin"],
                                 individual[ind_key]["2DBounds"]["ymin"],
                                 individual[ind_key]["2DBounds"]["xmax"],
                                 individual[ind_key]["2DBounds"]["ymax"]]
                    
                    bbox = fix_bounding_boxes(bbox_orig, max_val=display_img.shape)
                    
                    # only process an individual if its bounding box width and height are not zero
                    if bbox[2] - bbox[0] == 0 or bbox[3] - bbox[1] == 0:
                        continue


                    contours_lowpoly = []

                    #try:
                    ID_mask = cv2.inRange(seg_img, np.array([0, 0, ind_ID - 0]), np.array([0, 0, ind_ID + 0]))
                    indivual_occupancy = cv2.countNonZero(ID_mask)

                    # the kernel size for both dilation and median blur are to be determined by the bbounding boxes relative size
                    rel_size = ((bbox[2] - bbox[0]) / display_img.shape[0] + (bbox[3] - bbox[1]) / display_img.shape[0]) / 2
                    # values range from 0 (tiny) to 1 (huge)
                    # required smoothing 5 to 95
                    rel_size_root = int(round((15 * rel_size)/2.)*2 + 1) # round to next odd integer
                    #print("img:", i, "individual:", im, "rel_size", rel_size, rel_size_root)

                    # to simplify the generated masks and counter compression artifacts the original mask is dilated
                    # https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html
                    kernel = np.ones((rel_size_root, rel_size_root), 'uint8')
                    ID_mask_dilated = cv2.dilate(ID_mask, kernel, iterations=1)
                    # use median blur to further smooth the edges of the binary mask
                    ID_mask_dilated = cv2.medianBlur(ID_mask_dilated,rel_size_root)

                    # pad segmentation subwindow to prevent contours from being cut off

                    if mask_padding != 0:
                        ID_mask_dilated_padded = np.zeros([ID_mask_dilated.shape[0] + mask_padding * 2 , ID_mask_dilated.shape[1] + mask_padding * 2], 'uint8')
                        ID_mask_dilated_padded[mask_padding:-mask_padding,mask_padding:-mask_padding] = ID_mask_dilated
                        ID_mask_dilated = ID_mask_dilated_padded

                    


                    if DEBUG:
                        print("\nindividual",im,ID_mask_dilated.dtype)
                        print(hierarchy)
                        # draw the contours on the empty image
                        seg_img_display = seg_img.copy()
                        cv2.imshow("mask: ", ID_mask_dilated)
                        cv2.drawContours(seg_img_display[bbox[1]:bbox[3],bbox[0]:bbox[2]], contours, -1, (255,0,0), 3)
                        cv2.imshow("segmentation: ", seg_img_display[bbox[1]:bbox[3],bbox[0]:bbox[2]])
                        cv2.waitKey(0)


                    #indivual_occupancy = np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 - 1)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 + 1)]).all(axis = 2))
                    bbox_area = abs((bbox[2] - bbox[0]) * (bbox[3] - bbox[1])) + 1
                    bbox_occupancy = indivual_occupancy / bbox_area
                    #print("Individual", individual[0], "with bounding box occupancy ",bbox_occupancy)

                    if not enforce_single_class:
                        class_ID = subject_classes[colony['Subject Variations'][ind_key]["Class"].replace(" ","_")] + 1
                    else:
                        # here we use a single class, otherwise this can be replaced by size / scale values
                        class_ID = 1 # as the background = 0
                        
                    #cv2.putText(display_img, "ID: " + str(int(individual[0])), (bbox[0] + 10,bbox[3] - 10), font, fontScale, fontColor, lineType)
                    if bbox_occupancy > visibility_threshold:
                        #output_mask += np.array((ID_mask_dilated/255) * class_ID, np.uint8)
                        """
                        segmentation code goes here
                        - get mask for every individual
                        - set all values in out_mask = class_ID which correspond to the individual
                        """
                        
                        output_mask = np.where(ID_mask_dilated > 0, 
                                               np.array((ID_mask_dilated/255) * (class_ID), np.uint8),
                                               output_mask)
                        """
                        output_mask = np.where(ID_mask_dilated > 0, 
                                               ID_mask_dilated,
                                               output_mask)
                        """
                    else:
                        pass

            # uncomment to show resulting bounding boxes and masks
            if DEBUG:
                cv2.imshow("segmentation: " ,cv2.resize(seg_img_display, (int(seg_img.shape[1] / 2), 
                                                                  int(seg_img.shape[0] / 2))))
                cv2.imshow("labeled image", cv2.resize(display_img, (int(display_img.shape[1] / 2), 
                                                                     int(display_img.shape[0] / 2))))
                cv2.waitKey(1)
            
            img_name = img.split('/')[-1][:-4] + "_synth"
            
            """
            if img_name in files_train:
                img_out_path = target_dir + "/images/train/" + img_name + ".jpg"
                mask_out_path = target_dir + "/annotations/train/" + img_name + "_labelTrainIds.png"
            else:
                img_out_path = target_dir + "/images/test/" + img_name + ".jpg"
                mask_out_path = target_dir + "/annotations/test/" + img_name + "_labelTrainIds.png"
            """
            img_out_path = target_dir + "/images/" + img_name + ".jpg"
            mask_out_path = target_dir + "/annotations/" + img_name + "_labelTrainIds.png"
                
            cv2.imwrite(img_out_path, display_img)
            Image.fromarray(output_mask).save(mask_out_path, 'PNG')
            print("Saved", img_name)
            
        else:
            queueLock.release()
            
# setup as many threads as there are (virtual) CPUs
exitFlag_export = 0
if DEBUG:
    threadList_export = createThreadList(1)
else:
    threadList_export = createThreadList(getThreads())
print("Using", len(threadList_export), "threads for export...")
queueLock = threading.Lock()

# define paths to all images and set the maximum number of items in the queue equivalent to the number of images
workQueue_export = queue.Queue(len(dataset_img))
threads = []
threadID = 1

np.random.seed(seed=1)
ID_colours = np.random.randint(255, size=(255, 3))

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
lineType = 2

# we can additionally plot the points in the data files to check joint locations
plot_joints = True

# remember to refine an export folder when saving out your dataset
generate_dataset = True

def fix_bounding_boxes(coords,max_val = [1024,1024]):
    # fix bounding box coordinates so they do not reach beyond the image
    fixed_coords = []
    for c, coord in enumerate(coords):
        if c == 0 or c == 2:
            max_val_temp = max_val[0]
        else:
            max_val_temp = max_val[1]
            
        if coord >= max_val_temp:
            coord = max_val_temp
        elif coord <= 0:
            coord = 0
        
        fixed_coords.append(int(coord))
        
    return fixed_coords

timer = time.time()

# Create new threads
for tName in threadList_export:
    thread = exportThread(threadID, tName, workQueue_export)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue with samples
queueLock.acquire()
for i, (data, img, ID) in enumerate(zip(dataset_data , dataset_img, dataset_ID)):
    workQueue_export.put([i, data, img, ID])
queueLock.release()

# Wait for queue to empty
while not workQueue_export.empty():
    pass

# Notify threads it's time to exit
exitFlag_export = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print("Exiting Main export Thread")

# close all windows if they were opened
cv2.destroyAllWindows()

print("Total time elapsed:",time.time()-timer,"seconds")

Using 28 threads for export...
Starting Thread_0
Starting Thread_1
Starting Thread_2
Starting Thread_3
Starting Thread_4
Starting Thread_5
Starting Thread_6
Starting Thread_7
Starting Thread_8
Starting Thread_9
Starting Thread_10
Starting Thread_11
Starting Thread_12
Starting Thread_13
Starting Thread_14
Starting Thread_15
Starting Thread_16
Starting Thread_17
Starting Thread_18
Starting Thread_19
Starting Thread_20
Starting Thread_21
Starting Thread_22
Starting Thread_23
Starting Thread_24
Starting Thread_25
Starting Thread_26
Starting Thread_27
Saved SavedSavedSaved Leptoglossus-plants_00003_synth 
 Leptoglossus-plants_00005_synth
SavedLeptoglossus-plants_00022_synthLeptoglossus-plants_00001_synthSaved

Saved  Leptoglossus-plants_00017_synth
 Leptoglossus-plants_00024_synth
Leptoglossus-plants_00008_synthSaved 
Leptoglossus-plants_00025_synth
SavedSaved Leptoglossus-plants_00014_synth
 Leptoglossus-plants_00007_synth
Saved Leptoglossus-plants_00023_synth
Saved Leptoglossus-plants_000

Saved Leptoglossus-plants_00198_synth
Saved Leptoglossus-plants_00184_synth
Saved Leptoglossus-plants_00190_synth
Saved Leptoglossus-plants_00214_synth
Saved SavedLeptoglossus-plants_00191_synth Saved Leptoglossus-plants_00192_synthLeptoglossus-plants_00215_synth

Saved
Saved  Leptoglossus-plants_00202_synth
Leptoglossus-plants_00193_synth
Saved Leptoglossus-plants_00217_synth
Saved Leptoglossus-plants_00185_synth
Saved Leptoglossus-plants_00218_synth
Saved Leptoglossus-plants_00216_synth
Saved Leptoglossus-plants_00207_synth
Saved Leptoglossus-plants_00211_synth
Saved Leptoglossus-plants_00221_synth
Saved Leptoglossus-plants_00213_synth
Saved Leptoglossus-plants_00226_synth
Saved Leptoglossus-plants_00225_synth
Saved Leptoglossus-plants_00212_synthSaved 
Leptoglossus-plants_00210_synth
Saved Leptoglossus-plants_00219_synth
Saved Leptoglossus-plants_00227_synth
Saved Leptoglossus-plants_00230_synth
Saved Leptoglossus-plants_00234_synth
Saved SavedLeptoglossus-plants_00220_synth 
Leptog

Saved Leptoglossus-plants_00423_synth
Saved Leptoglossus-plants_00429_synth
Saved Leptoglossus-plants_00426_synth
Saved Leptoglossus-plants_00428_synth
Saved Saved Leptoglossus-plants_00438_synth
Leptoglossus-plants_00433_synth
SavedSaved  Leptoglossus-plants_00436_synthLeptoglossus-plants_00432_synth

Saved Leptoglossus-plants_00439_synth
Saved Leptoglossus-plants_00443_synth
Saved Leptoglossus-plants_00435_synth
Saved Leptoglossus-plants_00442_synth
SavedSaved Leptoglossus-plants_00451_synth
 Leptoglossus-plants_00453_synth
Saved Leptoglossus-plants_00441_synth
Saved Leptoglossus-plants_00452_synthSaved 
SavedLeptoglossus-plants_00434_synth
 Leptoglossus-plants_00458_synthSaved
 Leptoglossus-plants_00459_synthSaved 
Leptoglossus-plants_00466_synth
Saved Leptoglossus-plants_00463_synthSaved
 Leptoglossus-plants_00454_synth
Saved Leptoglossus-plants_00445_synth
Saved Leptoglossus-plants_00444_synth
SavedSaved Leptoglossus-plants_00450_synth Leptoglossus-plants_00462_synth

Saved Leptog

Saved Leptoglossus-plants_00641_synth
Saved Leptoglossus-plants_00653_synthSaved
 SavedLeptoglossus-plants_00651_synth
 Leptoglossus-plants_00650_synth
Saved Leptoglossus-plants_00657_synth
Saved Leptoglossus-plants_00668_synth
SavedSaved Leptoglossus-plants_00666_synth
 Leptoglossus-plants_00663_synth
Saved Leptoglossus-plants_00644_synth
Saved Leptoglossus-plants_00643_synth
SavedSaved Saved Leptoglossus-plants_00659_synthSaved Leptoglossus-plants_00631_synth
 Leptoglossus-plants_00652_synthLeptoglossus-plants_00649_synth


Saved Leptoglossus-plants_00660_synth
Saved Leptoglossus-plants_00673_synth
Saved Leptoglossus-plants_00656_synth
Saved Leptoglossus-plants_00671_synth
SavedSaved  Leptoglossus-plants_00674_synthLeptoglossus-plants_00675_synth

SavedSavedSaved  Leptoglossus-plants_00679_synth Leptoglossus-plants_00681_synth

SavedLeptoglossus-plants_00677_synthSaved 
 Leptoglossus-plants_00658_synth
Leptoglossus-plants_00667_synth
Saved Leptoglossus-plants_00665_synth
Saved Leptog

Saved Leptoglossus-plants_00866_synth
Saved Leptoglossus-plants_00863_synth
Saved Leptoglossus-plants_00886_synthSaved Leptoglossus-plants_00876_synth

Saved Leptoglossus-plants_00869_synth
SavedSavedSaved Leptoglossus-plants_00864_synth
SavedSaved  Saved Leptoglossus-plants_00880_synth
  Leptoglossus-plants_00871_synthLeptoglossus-plants_00877_synth
Leptoglossus-plants_00879_synth

Leptoglossus-plants_00873_synth
Saved Leptoglossus-plants_00867_synth
Saved Leptoglossus-plants_00872_synth
SavedSaved  Leptoglossus-plants_00878_synthSavedLeptoglossus-plants_00885_synth

 Leptoglossus-plants_00865_synth
Saved Leptoglossus-plants_00875_synth
SavedSaved Leptoglossus-plants_00891_synth Leptoglossus-plants_00882_synth

SavedSaved  Leptoglossus-plants_00887_synthLeptoglossus-plants_00874_synth

SavedSaved Leptoglossus-plants_00889_synth
 Leptoglossus-plants_00883_synth
Saved Leptoglossus-plants_00884_synth
SavedSaved  Leptoglossus-plants_00888_synth
Leptoglossus-plants_00881_synth
Saved Saved 

...

Saved Leptoglossus_08902_synth
Saved Leptoglossus_08890_synth
Saved Saved Leptoglossus_08906_synthLeptoglossus_08908_synth

Saved Leptoglossus_08894_synth
Saved Leptoglossus_08907_synth
Saved Leptoglossus_08911_synth
Saved SavedLeptoglossus_08924_synth
 Leptoglossus_08913_synth
Saved Leptoglossus_08919_synth
Saved Leptoglossus_08920_synth
SavedSaved Leptoglossus_08918_synthSaved 
 Leptoglossus_08909_synthLeptoglossus_08912_synth

SavedSaved Leptoglossus_08915_synth
 Leptoglossus_08917_synth
SavedSaved Leptoglossus_08928_synth Leptoglossus_08929_synth

Saved SavedLeptoglossus_08926_synth Leptoglossus_08938_synth

SavedSaved  Leptoglossus_08931_synthLeptoglossus_08910_synth

Saved Leptoglossus_08916_synth
Saved Leptoglossus_08937_synth
Saved Leptoglossus_08932_synth
SavedSaved SavedLeptoglossus_08914_synth Leptoglossus_08945_synth

 SavedSaved  Leptoglossus_08943_synth
Leptoglossus_08940_synthLeptoglossus_08922_synth

Saved Leptoglossus_08921_synth
Saved Leptoglossus_08935_synth
Saved Le

Saved Leptoglossus_09167_synth
Saved Leptoglossus_09165_synth
Saved Leptoglossus_09169_synth
Saved Leptoglossus_09154_synth
SavedSaved Leptoglossus_09156_synth
 Leptoglossus_09157_synthSaved
Saved Leptoglossus_09168_synth
 Leptoglossus_09173_synth
Saved Leptoglossus_09175_synth
Saved Saved Leptoglossus_09182_synthSaved
 Leptoglossus_09187_synthLeptoglossus_09178_synth

Saved SavedLeptoglossus_09171_synth 
Leptoglossus_09174_synth
Saved Leptoglossus_09186_synth
SavedSaved  Leptoglossus_09185_synth
Leptoglossus_09176_synth
Saved SavedSaved Leptoglossus_09192_synth
Leptoglossus_09194_synth
Saved Leptoglossus_09188_synth
 Leptoglossus_09193_synth
Saved Leptoglossus_09191_synth
Saved Leptoglossus_09199_synth
Saved Saved Leptoglossus_09204_synthLeptoglossus_09195_synth

Saved Leptoglossus_09202_synthSaved
 Leptoglossus_09190_synth
Saved Leptoglossus_09203_synth
Saved Leptoglossus_09201_synth
Saved Leptoglossus_09210_synth
Saved Leptoglossus_09213_synth
Saved Leptoglossus_09216_synth
Saved Sa

Saved Leptoglossus_09427_synth
SavedSaved Leptoglossus_09460_synth
 SavedLeptoglossus_09457_synth
 Leptoglossus_09455_synth
Saved Leptoglossus_09440_synth
Saved Leptoglossus_09461_synth
SavedSaved  Leptoglossus_09462_synthLeptoglossus_09450_synth

Saved Leptoglossus_09431_synth
Saved Leptoglossus_09453_synth
Saved Leptoglossus_09446_synth
SavedSaved Leptoglossus_09458_synth 
SavedSavedSaved  Leptoglossus_09449_synth
Leptoglossus_09434_synth Leptoglossus_09435_synthLeptoglossus_09433_synth

Saved
 Saved Saved Leptoglossus_09459_synth
Leptoglossus_09437_synthLeptoglossus_09445_synthSaved

 Leptoglossus_09454_synth
SavedSaved  Leptoglossus_09444_synth
Leptoglossus_09452_synth
SavedSaved  Leptoglossus_09465_synth
Leptoglossus_09438_synth
Saved Leptoglossus_09468_synth
Saved Leptoglossus_09439_synth
Saved Leptoglossus_09463_synth
Saved Leptoglossus_09466_synth
SavedSaved Leptoglossus_09469_synth
 Leptoglossus_09470_synth
SavedSaved Leptoglossus_09477_synth
Saved Leptoglossus_09475_synth
 Le

SavedSaved  Leptoglossus_09699_synth
Leptoglossus_09707_synth
Saved Leptoglossus_09702_synth
Saved Leptoglossus_09704_synth
SavedSaved  Leptoglossus_09696_synthLeptoglossus_09708_synth

Saved Leptoglossus_09709_synth
SavedSaved Leptoglossus_09711_synth
 Leptoglossus_09724_synth
Saved Leptoglossus_09720_synth
Saved Saved Leptoglossus_09718_synthLeptoglossus_09725_synth

Saved Leptoglossus_09722_synth
Saved Leptoglossus_09733_synth
Saved Leptoglossus_09736_synthSaved
 Leptoglossus_09723_synth
Saved Leptoglossus_09727_synthSavedSavedSaved 
Leptoglossus_09715_synth  Leptoglossus_09734_synthLeptoglossus_09732_synth

Saved 
Leptoglossus_09719_synth
Saved Leptoglossus_09737_synth
Saved Leptoglossus_09717_synth
Saved Leptoglossus_09738_synth
Saved Leptoglossus_09729_synth
SavedSaved  Leptoglossus_09728_synth
Saved Leptoglossus_09721_synth
Leptoglossus_09739_synth
Saved Leptoglossus_09731_synth
SavedSaved Leptoglossus_09730_synth
 Leptoglossus_09710_synth
Saved Saved Leptoglossus_09735_synth
Le

Saved Leptoglossus_09965_synth
Saved Leptoglossus_09977_synth
Saved SavedSaved Leptoglossus_09963_synth
 Leptoglossus_09964_synthLeptoglossus_09979_synth
Exiting Thread_9Saved

Exiting Thread_19 
Leptoglossus_09970_synth
Exiting Thread_18
Saved Leptoglossus_09978_synth
Exiting Thread_2
Saved Leptoglossus_09967_synth
Exiting Thread_13
Saved Leptoglossus_09975_synth
Exiting Thread_27
Saved Leptoglossus_09984_synth
Exiting Thread_8
Saved Leptoglossus_09971_synth
Exiting Thread_7
Saved Leptoglossus_09980_synth
Exiting Thread_21
Saved Leptoglossus_09981_synth
Exiting Thread_6
Saved Leptoglossus_09976_synth
Exiting Thread_3
Saved Leptoglossus_09983_synth
Exiting Thread_14
Saved Leptoglossus_09987_synth
Exiting Thread_12
Saved Leptoglossus_09982_synth
Exiting Thread_23
Saved Leptoglossus_09990_synth
Exiting Thread_16
Saved Leptoglossus_09986_synth
Exiting Thread_26
Saved Leptoglossus_09991_synth
Exiting Thread_15
Saved Leptoglossus_09985_synth
Exiting Thread_24
Saved Leptoglossus_09988_synth
